##### 您将获得每日历史销售数据。任务是预测测试集在每个商店中出售的产品总数。请注意，商店和产品的清单每月都会略有变化。创建可以处理此类情况的可靠模型是挑战的一部分。


##### 档案说明：
##### sales_train.csv-训练集。2013年1月至2015年10月的每日历史数据。
##### test.csv-测试集。您需要预测这些商店和产品在2015年11月的销售额。
##### sample_submission.csv-格式正确的示例提交文件。
##### items.csv-有关项目/产品的补充信息。
##### item_categories.csv-   有关项目类别的补充信息。
##### shop.csv-有关商店的补充信息。


##### 资料栏位：
##### ID-  代表测试集中的（商店，商品）元组的ID
##### shop_id-商店的唯一标识符
##### item_id-产品的唯一标识符
##### item_category_id-项目类别的唯一标识符
##### item_cnt_day-销售的产品数量。您正在预测该指标的每月金额
##### item_price-商品的当前价格
##### 日期  -格式为dd / mm / yyyy的日期
##### date_block_num-连续的月份号，为方便起见。2013年1月为0,2013年2月为1，...，2015年10月为33
##### item_name-  项目名称
##### shop_name-商店名称
##### item_category_name-项目类别名称

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from itertools import product

In [20]:
df_test = pd.read_csv('test.csv').copy()
df_train = pd.read_csv('sales_train.csv').copy()
items = pd.read_csv('items.csv').copy()
# item_categories = pd.read_csv('item_categories.csv').copy()
# shops = pd.read_csv('shops.csv').copy()
# sample_submission = pd.read_csv('sample_submission.csv').copy()

In [3]:
df_train.count()

NameError: name 'df_train' is not defined

# 特征工程

In [3]:
#删除异常数据
train_data = df_train.drop(df_train[df_train.item_price < 0].index | df_train[df_train.item_price >= 100000].index)
train_data = df_train.drop(df_train[df_train.item_cnt_day < 0].index | df_train[df_train.item_cnt_day >= 1000].index)

train_data1 = pd.pivot_table(train_data, index=['shop_id','item_id','date_block_num'], values=['item_cnt_day'], aggfunc=[np.sum],fill_value=0).reset_index();
train_data1.columns = train_data1.columns.droplevel().map(str)
train_data1 = train_data1.reset_index(drop=True).rename_axis(None, axis=1)
train_data1.columns.values[0]="shop_id";
train_data1.columns.values[1]="item_id";
train_data1.columns.values[2]="month_id";
train_data1.columns.values[3]="item_cnt_month";

train_data1

,shop_id,item_id,month_id,item_cnt_month
0,0,30,1,31
1,0,31,1,11
2,0,32,0,6
3,0,32,1,10
4,0,33,0,3
...,...,...,...,...
1608220,59,22164,27,2
1608221,59,22164,30,1
1608222,59,22167,9,1
1608223,59,22167,11,2


In [5]:
matrix = []
cols = ['month_id','shop_id','item_id']
for i in range(34):
    sales = train_data[train_data.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['month_id'] = matrix['month_id'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols, inplace=True)
print(matrix.info())

train_data1 = pd.pivot_table(train_data, index=['shop_id','item_id','date_block_num'], values=['item_cnt_day'], aggfunc=[np.sum],
fill_value=0).reset_index();
train_data1.columns = train_data1.columns.droplevel().map(str)
train_data1 = train_data1.reset_index(drop=True).rename_axis(None, axis=1)
train_data1.columns.values[0]="shop_id";
train_data1.columns.values[1]="item_id";
train_data1.columns.values[2]="month_id";
train_data1.columns.values[3]="item_cnt_month";

#合并matrix和train_data1数据
train_data1 = pd.merge(matrix, train_data1, on=["shop_id", "item_id", "month_id"], how='left')
train_data1['item_cnt_month'] = (train_data1['item_cnt_month'].fillna(0).astype(np.float16))
train_data1["month_id1"] = train_data1["month_id"] % 12;
print(train_data1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10884554 entries, 139168 to 10741269
Data columns (total 3 columns):
month_id    int8
shop_id     int8
item_id     int16
dtypes: int16(1), int8(2)
memory usage: 124.6 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10884554 entries, 0 to 10884553
Data columns (total 5 columns):
month_id          int8
shop_id           int8
item_id           int16
item_cnt_month    float16
month_id1         int8
dtypes: float16(1), int16(1), int8(3)
memory usage: 155.7 MB
None


In [57]:
item_cnt.isnull().sum()

item_id              0
month_id             0
item_total_month1    0
dtype: int64

In [14]:
train_data2 = pd.pivot_table(df_train, index=['shop_id','item_id','date_block_num'], values=['item_cnt_day'], aggfunc=[np.sum],
fill_value=0).reset_index();
train_data2.columns = train_data2.columns.droplevel().map(str)
train_data2 = train_data2.reset_index(drop=True).rename_axis(None, axis=1)
train_data2.columns.values[0]="shop_id";
train_data2.columns.values[1]="item_id";
train_data2.columns.values[2]="month_id";
train_data2.columns.values[3]="item_cnt_month1";
train_data2["month_id"] = train_data2["month_id"] + 1;

item_price1 = pd.pivot_table(df_train, index=['item_id','date_block_num'], values=['item_price'], aggfunc=[np.mean], fill_value=0).reset_index();
item_price1.columns = item_price1.columns.droplevel().map(str)
item_price1 = item_price1.reset_index(drop=True).rename_axis(None, axis=1)
item_price1.columns.values[0]="item_id";
item_price1.columns.values[1]="month_id";
item_price1.columns.values[2]="item_mean_price1";
item_price1["month_id"] = item_price1["month_id"] + 1;

shop_cnt = pd.pivot_table(train_data1, index=['shop_id','month_id'], values=['item_cnt_month'], aggfunc=[np.mean], fill_value=0).reset_index();
shop_cnt.columns = shop_cnt.columns.droplevel().map(str)
shop_cnt = shop_cnt.reset_index(drop=True).rename_axis(None, axis=1)
shop_cnt.columns.values[0]="shop_id";
shop_cnt.columns.values[1]="month_id";
shop_cnt.columns.values[2]="shop_cnt_month1";
shop_cnt["month_id"] = shop_cnt["month_id"] + 1;

item_cnt = pd.pivot_table(train_data1, index=['item_id','month_id'], values=['item_cnt_month'], aggfunc=[np.mean], fill_value=0).reset_index();
item_cnt.columns = item_cnt.columns.droplevel().map(str)
item_cnt = item_cnt.reset_index(drop=True).rename_axis(None, axis=1)
item_cnt.columns.values[0]="item_id";
item_cnt.columns.values[1]="month_id";
item_cnt.columns.values[2]="item_total_month1";
item_cnt["month_id"] = item_cnt["month_id"] + 1;

In [21]:
combined_data = train_data1.append(df_test);
#合并上述的五个特征到训练集中
combined_data = pd.merge(combined_data, train_data2, on = ['shop_id', 'item_id','month_id'], how = 'left')
combined_data["item_cnt_month1"].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, item_price1, on = ['item_id','month_id'], how = 'left')
combined_data["item_mean_price1"].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, shop_cnt, on = ['shop_id','month_id'], how = 'left')
combined_data["shop_cnt_month1"].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, item_cnt, on = ['item_id', 'month_id'], how = 'left')
combined_data["item_total_month1"].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, items, on = ['item_id'], how = 'left')
combined_data["item_category_id"].fillna(0, inplace=True)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [61]:
combined_data = combined_data.drop(['item_name', 'item_cnt_month'],axis=1)
combined_data.isnull().sum()
# combined_data

ID                   10884554
item_id                     0
month_id               214200
month_id1              214200
shop_id                     0
item_cnt_month1             0
item_mean_price1            0
shop_cnt_month1             0
item_total_month1           0
item_category_id            0
dtype: int64

# 训练模型

In [72]:
train = combined_data[combined_data.ID.isnull()].drop('ID', axis=1)
test = combined_data[combined_data.ID.notnull()].drop(['ID','item_cnt_month1'], axis=1)
X = train.drop('item_cnt_month1', axis=1)
y = train['item_cnt_month1']

In [76]:
test

,item_id,month_id,month_id1,shop_id,item_mean_price1,shop_cnt_month1,item_total_month1,item_category_id
10884554,5037,NaN,NaN,5,0.0,0.0,0.0,19
10884555,5320,NaN,NaN,5,0.0,0.0,0.0,55
10884556,5233,NaN,NaN,5,0.0,0.0,0.0,19
10884557,5232,NaN,NaN,5,0.0,0.0,0.0,23
10884558,5268,NaN,NaN,5,0.0,0.0,0.0,20
...,...,...,...,...,...,...,...,...
11098749,18454,NaN,NaN,45,0.0,0.0,0.0,55
11098750,16188,NaN,NaN,45,0.0,0.0,0.0,64
11098751,15757,NaN,NaN,45,0.0,0.0,0.0,55
11098752,19648,NaN,NaN,45,0.0,0.0,0.0,40


In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [79]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'max_depth': [4],
        'n_estimators': [50],
        'learning_rate': [0.1]
#         ,'subsample': [0.8]
    }
]
gb_reg = GradientBoostingRegressor()
grid_search2 = GridSearchCV(gb_reg, param_grid, verbose=2, cv=3) # cv是交叉验证分成多少分
grid_search2.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] learning_rate=0.1, max_depth=4, n_estimators=50 .................
[CV] .. learning_rate=0.1, max_depth=4, n_estimators=50, total=18.8min
[CV] learning_rate=0.1, max_depth=4, n_estimators=50 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.8min remaining:    0.0s


[CV] .. learning_rate=0.1, max_depth=4, n_estimators=50, total=19.9min
[CV] learning_rate=0.1, max_depth=4, n_estimators=50 .................
[CV] .. learning_rate=0.1, max_depth=4, n_estimators=50, total=20.4min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 59.1min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change=None,
                

In [80]:
grid_search2.best_score_

0.7042695977708084

In [81]:
grid_search2.score(X_test, y_test)

0.7456255908470076

In [102]:
gb_reg = grid_search2.best_estimator_
test['month_id'] = test['month_id'].fillna(34)
test['month_id1'] = test['month_id1'].fillna(34%12)
test
predict = gb_reg.predict(test)
predict
csv = {'Id':df_test['ID'], 'item_cnt_month':predict}
df =pd.DataFrame(csv)
df.to_csv('predcit5-29-1.csv',index=None)

In [100]:
train['month_id'].max()

33.0